# Lecture 3 — Improving the Resume Scorer

In [ ]:
# Add current directory to Python path for imports
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd()))

import json
import random
import pandas as pd
from resume_utils import load_resumes, load_job_requirements, structured_llm_call

random.seed(111)

# Configuration
OPENROUTER_API_KEY = ""  # Paste your key here

if not OPENROUTER_API_KEY or OPENROUTER_API_KEY.strip() == "":
    raise RuntimeError(
        "⚠️  Please set OPENROUTER_API_KEY above before running this notebook.\n"
        "Get your key from: https://openrouter.ai/keys"
    )

print("✓ Imports loaded")
print("✓ API key configured")

## Load Data and Sample Resumes

Load resume data and randomly sample a subset for analysis.

In [ ]:
resumes = load_resumes('../data/resumes_final.csv')
job_req = load_job_requirements('../data/job_req_senior.md')

print(f"Loaded {len(resumes)} resumes")

resume_list = list(resumes.values())
resume_samples = random.sample(resume_list, 3)
print(f"Selected {len(resume_samples)} resumes for analysis")

## Baseline: Monolithic Approach

Start with a single prompt that does everything:
- Reads the resume
- Compares it to job requirements
- Outputs a 0-100 score

This is our baseline to compare against improved approaches.

In [ ]:
# Baseline: Single prompt that does everything
baseline_prompt = """
Score this resume against the job requirements on a 0-100 scale.

Consider:
- Years of experience required (5-10 years)
- Technical skills match
- Education level
- Overall fit

Provide a final score.
"""

baseline_schema = {
    "score": "number (0-100)",
    "reasoning": "string explaining the score"
}

# Collect results for all resumes
results = []

for idx, resume in enumerate(resume_samples):
    print(f"{idx+1} of {len(resume_samples)}")
    baseline_result = structured_llm_call(
        api_key=OPENROUTER_API_KEY,
        prompt=baseline_prompt,
        context_data={
            "resume": resume['Resume_str'],
            "job_requirements": job_req
        },
        output_schema=baseline_schema,
        temperature=0.3
    )
    
    results.append({
        'resume_id': resume['ID'],
        'score': baseline_result['result'].get('score', None),
        'tokens': baseline_result['usage'].get('total_tokens', 0),        
        'reasoning': baseline_result['result'].get('reasoning', ''),
    })

# Create DataFrame from results
monolith_scores_df = pd.DataFrame(results)

print(3*'\n')
print(monolith_scores_df)

---

## TODO 1: Decompose - Extract Years of Experience

**Goal**: Break down the monolithic approach by extracting specific features first.

**Your Task**:
1. Create a prompt that extracts ONLY years of experience
2. Require evidence/citations from the resume
3. Define an output schema for the extraction
4. Use `structured_llm_call` to run it on all resume samples

In [ ]:
# TODO 1: Write your extraction prompt here
# Hints:
# - Focus ONLY on extracting years of experience
# - Require citations/evidence from the resume
# - Be specific about what counts as "experience"

number_of_years_extraction_prompt = """
WRITE YOUR OWN PROMPT AND SEE IF IT WORKS.
"""

number_of_years_schema = {
    "years": "number (0-100)",
    "reasoning": "string explaining how you calculated the years, with citations"
}

results = []
for idx, resume in enumerate(resume_samples):
    print(f"{idx+1} of {len(resume_samples)}") 
    
    year_result = structured_llm_call(
        api_key=OPENROUTER_API_KEY,
        prompt=number_of_years_extraction_prompt,
        context_data={
            "resume": resume['Resume_str'],
            "job_requirements": job_req
        },
        output_schema=number_of_years_schema,
        temperature=0.3
        )

    results.append({
        'resume_id': resume['ID'],
        'years_working': year_result['result'].get('years', None),
        'tokens': year_result['usage'].get('total_tokens', 0),        
        'reasoning': year_result['result'].get('reasoning', ''),
    })


# Create DataFrame from results
work_experience_scores = pd.DataFrame(results)

print(3*'\n')
print(work_experience_scores)

---

## TODO 2: Extract and Score Relevant Technologies

**Goal**: Extract technology skills from resumes and score them against job requirements.

**Your Task**:
1. Copy the previous cell and update it to extract technologies/skills from the resume
2. Compare extracted technologies against the required technologies in the job requirements
3. Provide a score (0-100) with clear reasoning based on technology match


In [ ]:
# TODO 2: Copy the work experience extraction cell above and modify it for technology extraction
# Hints:
# - Extract technologies/skills mentioned in the resume
# - Compare against required technologies from job_req
# - Score based on match percentage (0-100)
# - Use a similar structure: prompt, schema, loop, DataFrame

# Your code here:
# 1. Define technology_extraction_prompt
# 2. Define technology_schema (should include: technologies_found, required_technologies, match_score, reasoning)
# 3. Loop through resume_samples
# 4. Create technology_scores DataFrame


---

## TODO 3: Combine and Compare

**Goal**: Compare the scores from the monolithic approach versus your decomposed approach.

**Your Task**:
- Merge the year data (`work_experience_scores`) with the technology scores
- Create a composite score based on the extracted features
- Compare your composite score against the monolithic 0-100 scores
- Analyze the differences: which approach is more consistent? Which provides better explanations?

---

## TODO 4: Extensions

**Goal**: Add more sophisticated feature extraction and scoring.

**Ideas to explore**:
1. **Education Requirements**
   - Extract degree level and field
   - Check if it meets "Bachelor's in CS or equivalent"
   - Score based on education match

2. **Leadership/Mentoring**
   - Look for evidence of mentoring junior developers
   - Check for team lead or senior roles
   - Score based on leadership experience

3. **Multi-Resume Comparison**
   - Run your improved scorer on 20 resumes
   - Rank candidates by composite score
   - Compare token costs: baseline vs. decomposed approach

Pick one or more to implement!